In [1]:
import numpy as np
from time import time
from __future__ import print_function
import tensorflow as tf

In [2]:
# Load and parse the data (N instances, D features, L=6 labels)
XY = np.genfromtxt('data/scene.csv', skip_header=1, delimiter=",")
N,DL = XY.shape
L = 6
D = DL - L
Y = XY[:,0:L].astype(int)
X = XY[:,L:D+L]

# Split into train/test sets
n = int(N*6/10)
X_train = X[0:n]
Y_train = Y[0:n]
X_test = X[n:]
Y_test = Y[n:]

In [3]:
class Network():

    def __init__(self, learning_rate=0.005):
        ''' initialize the classifier with default (best) parameters '''
        # TODO
        self.alpha = learning_rate
        self.weight_decay = 3e-4
        self.batch_size = 30
        self.num_epoch = 0
        self.clock = time()

        # tf Graph Input
        self.x = tf.placeholder(tf.float32, [None, 294]) 
        self.y = tf.placeholder(tf.float32, [None, 6]) 

        # Set model weights
        W1 = tf.Variable(tf.random_normal([294, 64]))
        b1 = tf.Variable(tf.random_normal([64]))
        W2 = tf.Variable(tf.random_normal([64, 32]))
        b2 = tf.Variable(tf.random_normal([32]))
        W3 = tf.Variable(tf.random_normal([32, 6]))
        b3 = tf.Variable(tf.random_normal([6]))

        # Construct model
        layer1 = tf.nn.sigmoid(tf.matmul(self.x, W1) + b1)
        layer2 = tf.nn.sigmoid(tf.matmul(layer1, W2) + b2)
        self.pred = tf.add(tf.matmul(layer2, W3), b3) 

        # Minimize error using sigmoid cross entropy
        self.cost = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                labels=self.y,
                logits=self.pred)) + self.weight_decay*tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()])

        # Gradient Descent
        optimizer = tf.train.AdamOptimizer(learning_rate=self.alpha)
        self.opt = optimizer.minimize(self.cost)

        # Initialize the variables (i.e. assign their default value)
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)


    def fit(self,X,Y,warm_start=True,n_epochs=10):
        ''' train the network, and if warm_start, then do not reinit. the network
            (if it has already been initialized)
        '''

        self.n_labels = Y.shape[1]
        self.batch_per_epoch = int(X.shape[0]/self.batch_size)

        if not warm_start:
            self.sess.close()
            self.__init__()

        # Training cycle
        for epoch in range(n_epochs):
            self.num_epoch += 1
            avg_cost = 0.

            # Loop over all batches
            for i in range(self.batch_per_epoch):
                batch_x = X[self.batch_size*i:self.batch_size*(i+1),:]
                batch_y = Y[self.batch_size*i:self.batch_size*(i+1),:] 

                # Run optimization op (backprop) and cost op (to get loss value)
                _, c = self.sess.run([self.opt, self.cost], 
                    feed_dict={self.x: batch_x, self.y: batch_y})

                # Compute average loss
                avg_cost += c / self.batch_per_epoch

            # Display logs per epoch step
            if self.num_epoch % a == 0:
                print("Epoch:", '%04d' % (self.num_epoch), 
                    "cost=", "{:.9f}".format(avg_cost), 
                    "time=", "{:.4f}".format(time()-self.clock))

        return self

    def predict_proba(self,X):
        ''' return a matrix P where P[i,j] = P(Y[i,j]=1), 
        for all instances i, and labels j. '''
        predict_proba = self.sess.run(tf.nn.sigmoid(self.pred), feed_dict={self.x: X})
        return predict_proba

    def predict(self,X):
        ''' return a matrix of predictions for X '''
        return (self.predict_proba(X) >= 0.4).astype(int)



In [4]:
# Test our classifier 
h = Network()

In [5]:
t0 = time()
while (time() - t0) < 15:
    h.fit(X_train,Y_train, warm_start=True)

print("Trained %d epochs in %d seconds." % (h.num_epoch,int(time() - t0)))

Epoch: 0010 cost= 0.853097139 time= 0.5702
Epoch: 0020 cost= 0.428178379 time= 0.8382
Epoch: 0030 cost= 0.322282086 time= 1.0972
Epoch: 0040 cost= 0.279268830 time= 1.3679
Epoch: 0050 cost= 0.258132236 time= 1.6366
Epoch: 0060 cost= 0.246414123 time= 1.8934
Epoch: 0070 cost= 0.239566639 time= 2.1537
Epoch: 0080 cost= 0.236606733 time= 2.4179
Epoch: 0090 cost= 0.238908311 time= 2.6954
Epoch: 0100 cost= 0.231450425 time= 2.9648
Epoch: 0110 cost= 0.229578841 time= 3.2255
Epoch: 0120 cost= 0.230678666 time= 3.4945
Epoch: 0130 cost= 0.227148386 time= 3.7650
Epoch: 0140 cost= 0.224203603 time= 4.0297
Epoch: 0150 cost= 0.223759547 time= 4.2961
Epoch: 0160 cost= 0.223134817 time= 4.7535
Epoch: 0170 cost= 0.223234596 time= 5.3151
Epoch: 0180 cost= 0.222828476 time= 5.6627
Epoch: 0190 cost= 0.222748855 time= 5.9223
Epoch: 0200 cost= 0.222755445 time= 6.2003
Epoch: 0210 cost= 0.223551487 time= 6.4617
Epoch: 0220 cost= 0.226538351 time= 6.7230
Epoch: 0230 cost= 0.223137301 time= 6.9864
Epoch: 0240

In [6]:
def results(X_test):
    proba = h.predict_proba(X_test)
    Y_pred = (proba >= 0.4).astype(int)
    print("sum(sum(Y_pred)) = {}".format(sum(sum(Y_pred))))
    print("sum(sum(Y_test)) = {}".format(sum(sum(Y_test))))
    tester = Y_pred != Y_test
    loss = np.mean(tester)
    print("Hamming loss     =", loss)

    for i in range(len(tester)):
        if any(tester[i]):
            formattedProba = [ '%.2f' % elem for elem in proba[i] ]
            print("{:03d} : PROB = {} | PRED = {}  |  TEST = {}".format(i, formattedProba, Y_pred[i],Y_test[i]))

In [7]:
results(X_test)

sum(sum(Y_pred)) = 831
sum(sum(Y_test)) = 859
Hamming loss     = 0.08125
002 : PROB = ['0.02', '0.01', '0.00', '0.93', '0.35', '0.02'] | PRED = [0 0 0 1 0 0]  |  TEST = [0 0 0 1 1 0]
003 : PROB = ['0.57', '0.00', '0.00', '0.01', '0.11', '0.49'] | PRED = [1 0 0 0 0 1]  |  TEST = [1 0 0 0 0 0]
006 : PROB = ['0.00', '0.00', '0.01', '0.57', '0.18', '0.10'] | PRED = [0 0 0 1 0 0]  |  TEST = [0 0 0 0 1 0]
007 : PROB = ['0.00', '0.06', '0.01', '0.03', '0.49', '0.07'] | PRED = [0 0 0 0 1 0]  |  TEST = [0 0 0 1 0 0]
008 : PROB = ['0.42', '0.02', '0.05', '0.00', '0.01', '0.55'] | PRED = [1 0 0 0 0 1]  |  TEST = [1 0 0 0 0 0]
009 : PROB = ['0.81', '0.00', '0.00', '0.02', '0.02', '0.49'] | PRED = [1 0 0 0 0 1]  |  TEST = [1 0 0 0 0 0]
010 : PROB = ['0.01', '0.00', '0.00', '0.01', '0.85', '0.30'] | PRED = [0 0 0 0 1 0]  |  TEST = [0 0 0 1 0 0]
012 : PROB = ['0.35', '0.02', '0.63', '0.00', '0.02', '0.02'] | PRED = [0 0 1 0 0 0]  |  TEST = [1 0 0 0 0 0]
013 : PROB = ['0.12', '0.01', '0.00', '0.01', '

788 : PROB = ['0.00', '0.03', '0.00', '0.13', '0.25', '0.19'] | PRED = [0 0 0 0 0 0]  |  TEST = [0 0 0 1 1 0]
792 : PROB = ['0.01', '0.08', '0.00', '0.00', '0.61', '0.51'] | PRED = [0 0 0 0 1 1]  |  TEST = [0 0 0 0 0 1]
794 : PROB = ['0.46', '0.00', '0.00', '0.00', '0.69', '0.15'] | PRED = [1 0 0 0 1 0]  |  TEST = [0 0 0 0 1 0]
797 : PROB = ['0.65', '0.00', '0.00', '0.17', '0.35', '0.03'] | PRED = [1 0 0 0 0 0]  |  TEST = [0 0 0 0 1 0]
798 : PROB = ['0.28', '0.00', '0.00', '0.03', '0.09', '0.37'] | PRED = [0 0 0 0 0 0]  |  TEST = [1 0 0 0 0 0]
